# <center> ACC_Baroclinic_Barotropic </center>

***
**ACC Transport:**

The ACC is set by a number of factors: Wind Stress ($\tau$), Surface buoyancy forcing and remote forcing altering stratification:
Transport given by:
\begin{equation}
T=\iint u \mathrm{d}z\mathrm{d}y
\end{equation}
If you substitute thermal wind into the above equation where thermal wind is given as:
\begin{equation}
u_z=\frac{u_z}{f\rho_0}\rho_y
\end{equation}
Transport can be given by:
\begin{equation}
T=\frac{g}{f\rho_0}\iint \rho_N -\rho_S \, \mathrm{d}z^2
\end{equation}

Which looks like the ACC strength could be set by stratification. Hogg 2010 showed that you can drive the ACC without $\tau$. Marshall and Radko 2003 suggested transport depends on the wind stress and buoyancy difference. 

With no diabatic forcing form stress balances wind stress (Ward 2011) so $\tau\,=\,S_{bottom}$ where $S_{bottom}$ is independent of mean flow

When diabetic terms are taken into account the there may be a feedback with the meridional layer volume transport - feedbacks with ACC and overturning?

***

**What can my model tell me?**

* To what extent is the ACC controlled by stratification?

* If it is a significant control this will change in my model! My winds remian the same all that changes is stratification!!

* Do my transports match?

* This is over the ACC so where do you define the boundary of the ACC in the model? 

In [2]:
# Load Modules
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
sys.path.append('/noc/users/hb1g13/Python/python_functions/')
import useful as hb
from HB_Plot import nf, fmt
%matplotlib inline

In [ ]:
Year = 'TAV.nc'
# tau=['3','10','30','100','300','1000','3000','10000','Closed']
tau = ['3', '300', '3000', 'Closed']
Figletter = ['a) ','b) ','c) ','d) ']
x = '/noc/msm/scratch/students/hb1g13/Mobilis'

# Now Make file structure
check = 0
runs = []
for i in range(len(tau)):
    flist = x+'/'+str(tau[i])+'daynokpp/'+Year
    if not os.path.exists(flist):
        print ' WARNING: '+flist+' does not exist! (skipping this tau...)'
        check += 0
    else:
        check += 1
        runs.append(i) 
gridfilepath = x+'/'+str(tau[0])+'daynokpp/'
file2 = netCDF4.Dataset(gridfilepath+'grid.nc', 'r')
Zp = file2.variables['Zp1'][:]
Z = file2.variables['Z'][:]
Y = file2.variables['Yp1'][:]
Yc = file2.variables['Y'][:]
X=file2.variables['Xp1']
Xp=X[:]
Zmatrix = np.zeros((len(Z), len(Y)))
Zmatrix[:, :] = 1
dx = Y[1]-Y[0]  # Find Resolution
dz = Zp[0:len(Zp)-1]-Zp[1:len(Zp)]
Zmatrix = np.apply_along_axis(np.multiply, 0, Zmatrix, dz)
Runs=np.array(runs)

In [ ]:
# Constant Parameters
alpha=2.000000E-04
G=-9.81
rho_0=1000
C_p=3985
Res=5000
Ly=2000e3
Lx=1000e3 
H=2985 
nz=30 
f=-1.000000E-04
tref=4.0
dz=Zp[0:len(Zp)-1]-Zp[1:len(Zp)]

In [ ]:
T1=zeros(shape(Runs))
T2=zeros(shape(Runs))
tau=['3','10','30','100','300','1000','3000','10000','Closed']
for I in range(len(Runs)):
    filename=x+'/'+str(tau[Runs[I]])+'daynokpp/'+Year
    file2read = netcdf.NetCDFFile(filename,'r')
    U=file2read.variables["UVEL"]
    U=U[:]
    Temp=file2read.variables['THETA']
    Temp=Temp[:]*1
    Ubar=np.mean(U,axis=0)
    Uzone=np.nansum(Ubar,axis = 2)*5000
    dz=Zp[0:len(Zp)-1]-Zp[1:len(Zp)]
    # Got rid of for loop here (much quicker!!)
    psi2=np.apply_along_axis(np.multiply,0,Uzone,dz)
    psi=np.cumsum(psi2[::-1,:],axis=0)
    npad = ((0,1), (0,0))
    # Pad with zeros at bottom
    psi = np.pad(psi, pad_width=npad, mode='constant', constant_values=0)
    y =Y/1000
    Psi=psi/10**6
    T1[I]=np.max(Psi)
    #Udy=np.sum(Ubar*Res,axis=2)
    ACC=np.zeros((len(Z),len(Yc),len(Xp)))
    for ix in range(len(Yc)):
        for j in range(len(Xp)):
            for k in range(0,len(Z)-1):
                ACC[k,ix,j] = dz[k]*Ubar[k,ix,j]
    ACCf=np.cumsum(ACC[::-1,:,:],axis=0)
    ACCf[ACCf<0.15]=0 # only over ACC!
    ACC=np.mean(np.sum(ACCf[::-1,:,:],axis=1))
    #T1[I]=np.max(ACC)
    Tav=np.mean(Temp,axis=0)
    Tavlat=np.mean(Tav,axis=2)
    Dens=np.zeros(shape(Tavlat))
    Densdz=np.zeros(shape(Tavlat[:,1]))  
    R2=np.zeros(shape(Tavlat[:,1]))
    Dens=rho_0*(1-(alpha*(Tavlat-tref)))
    RhoN=np.mean(Dens[::-1,-120:-80],axis=1)
    RhoS=np.mean(Dens[::-1,80:120],axis=1)
    dz=Zp[0:len(Zp)-1]-Zp[1:len(Zp)]
    for i in range(len(Z)):
        Densdz[i]=(RhoN[i]-RhoS[i])*dz[i]
    R1=np.cumsum(Densdz)
    for i in range(len(Z)):
        R2[i]=R1[i]*dz[i]
    T2[I]=-G/(f*rho_0)*np.sum(R2)
